In [1]:
import numpy as np
import pandas as pd
import pickle

col_names = pd.read_csv('../data/dataset.csv', nrows=0).columns
dtype_map = {'class' : np.int16, 'bankruptcy_after_years' : np.int16}
dtype_map.update({col: np.float64 for col in col_names if col not in dtype_map})

df = pd.read_csv('../data/dataset.csv', dtype=dtype_map)
df = df.drop([df.columns[0], df.columns[1], df.columns[2], 'year'], axis=1)
df.drop_duplicates(keep=False, inplace=True)

df.head()

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class,bankruptcy_after_years
0,0.200550,0.37951,0.39641,2.0472,32.3510,0.38825,0.249760,1.33050,1.1389,0.50494,...,0.39718,0.87804,0.001924,8.4160,5.1372,82.658,4.4158,7.4277,0,0
1,0.209120,0.49988,0.47225,1.9447,14.7860,0.00000,0.258340,0.99601,1.6996,0.49788,...,0.42002,0.85300,0.000000,4.1486,3.2732,107.350,3.4000,60.9870,0,0
2,0.248660,0.69592,0.26713,1.5548,-1.1523,0.00000,0.309060,0.43695,1.3090,0.30408,...,0.81774,0.76599,0.694840,4.9909,3.9510,134.270,2.7185,5.2078,0,0
3,0.081483,0.30734,0.45879,2.4928,51.9520,0.14988,0.092704,1.86610,1.0571,0.57353,...,0.14207,0.94598,0.000000,4.5746,3.6147,86.435,4.2228,5.5497,0,0
4,0.187320,0.61323,0.22960,1.4063,-7.3128,0.18732,0.187320,0.63070,1.1559,0.38677,...,0.48431,0.86515,0.124440,6.3985,4.3158,127.210,2.8692,7.8980,0,0


XGBoost model and k-fold cross-validation, binary classification:

In [2]:
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

scoring = {'accuracy' : make_scorer(accuracy_score),
           'precision' : make_scorer(precision_score, average='micro'),
           'recall' : make_scorer(recall_score, average='micro'), 
           'f1_score' : make_scorer(f1_score, average='micro')}

   
    
def save_model(model, frs, label, out_dir):
    model.fit(frs, label)
    pickle.dump(model, open(out_dir,'wb'))


def validate(out_dir, frs, label):  
    model = pickle.load(open(out_dir,'rb'))   
    kfold = KFold(n_splits=5)
    return cross_validate(estimator=model, X=frs, y=label, cv=kfold, scoring=scoring)

Fit and save  XGBoost classifier with all features (original data with missing values)

In [4]:
all_frs = df[df.columns.difference(['bankruptcy_after_years', 'class'])] # with NaNs
label = df['bankruptcy_after_years']
save_model(xgboost.XGBClassifier(), all_frs, label, '../models/xgboost.pkl')

Check cross-validation accuracy, precision, recall, and f1-score for XGBoost classifier fitted on all features with kept NaNs:

In [46]:
results = validate('../models/xgboost.pkl', all_frs, label)
results

{'fit_time': array([144.80531359, 139.43467355, 174.6276567 , 159.52204514,
        128.61141706]),
 'score_time': array([0.21106172, 0.2135067 , 0.25859308, 0.23638296, 0.17466044]),
 'test_accuracy': array([0.96819995, 0.95306266, 0.99941328, 0.94191504, 0.89238352]),
 'test_precision': array([0.96819995, 0.95306266, 0.99941328, 0.94191504, 0.89238352]),
 'test_recall': array([0.96819995, 0.95306266, 0.99941328, 0.94191504, 0.89238352]),
 'test_f1_score': array([0.96819995, 0.95306266, 0.99941328, 0.94191504, 0.89238352])}

In [47]:
np.mean(results['test_accuracy'])

0.9509948928308628

In [48]:
np.mean(results['test_precision'])

0.9509948928308628

In [49]:
np.mean(results['test_recall'])

0.9509948928308628

In [50]:
np.mean(results['test_f1_score'])

0.9509948928308628

- As it is seen from the 5-fold cross validation results, XGBoost classifier performs well on non-reduced and highly skewed imbalanced data - it means that further data reduction and / or imputation are redundant steps. The dataset chosen for analysis already contains synthetic features, so engineering new ones is not necessary in case of such a high performance.

In [7]:
# from sklearn.metrics import precision_recall_curve, auc, average_precision_score, roc_curve
# import matplotlib as plt

# def plot_precision_recall(out_dir, y_test):
#     model = pickle.load(open(out_dir,'rb'))  
#     class_labels = np.sort(y_test.unique())

#     row_count = np.ceil(len(class_labels) / 3).astype(int)
#     fig, axes = plt.subplots(row_count, 3, figsize=(15, row_count*5))
#     axes = axes.flatten()
    
#     preds = model.predict_proba(all_frs)

#     if len(axes) > len(class_labels):
#         for i in range(len(class_labels), len(axes)):
#             fig.delaxes(axes[i])

#     for i, class_label in enumerate(class_labels):
#         axes[i].axhline(sum(y_test == class_label)/len(y_test), color='navy', lw=2, linestyle='--', label='baseline')
#         actuals = np.where(y_test == class_label, 1, 0)
#         predicted_probabilities = preds[:,i]
#         precision, recall, thresholds = precision_recall_curve(actuals, predicted_probabilities)
#         auc_score = auc(recall, precision)
#         ap_score = average_precision_score(actuals, predicted_probabilities)
#         axes[i].plot(recall, precision, lw=2, label=f"""AUC: {auc_score:.2}; AP : {ap_score:.2}""")

#         axes[i].legend()
#         axes[i].set_title(f'Precision-recall curve: class {class_label}')
#         axes[i].set_xlabel('Recall')
#         axes[i].set_ylabel('Precision')

#         axes[i].set_xlim(-0.05, 1.05)
#         axes[i].set_ylim(-0.05, 1.05)

#     return axes

    
# plot_precision_recall('../models/xgboost.pkl', label)
